In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.content_02 import get_per_sample_metrics

metrics_df = get_per_sample_metrics()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130766400 entries, 0 to 130766399
Data columns (total 18 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   seed            int64 
 1   checkpoint      object
 2   gazetteer_size  int64 
 3   error_ratio     int64 
 4   error_part      object
 5   timestep        int64 
 6   dataset         object
 7   doc_id          object
 8   targets         int64 
 9   entity_type     object
 10  tp              int64 
 11  fn              int64 
 12  fp              int64 
 13  error_type1     int64 
 14  error_type2     int64 
 15  error_type3     int64 
 16  error_type4     int64 
 17  error_type5     int64 
dtypes: int64(13), object(5)
memory usage: 17.5+ GB


## Performance table 1

Rows: Dimensions:  "gazetteer_size", "error_ratio", "error_part"
Columns: Timestep:

In [4]:
from evaluations.content_02 import aggregate_performance_metrics

agg_df = aggregate_performance_metrics(metrics_df).reset_index()
agg_df = agg_df[agg_df["dataset"]=="lowner_test"]
agg_df

,seed,gazetteer_size,error_ratio,error_part,timestep,dataset,fn,fp,tp,precision,recall,f1
1,1,2000,0,both,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
4,1,2000,0,both,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
7,1,2000,0,both,2,lowner_test,5826,4898,37890,88.552865,86.673072,87.602885
10,1,2000,0,gazetteer,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
13,1,2000,0,gazetteer,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,3,8000,15,gazetteer,1,lowner_test,6626,4930,37090,88.267492,84.843078,86.521415
1285,3,8000,15,gazetteer,2,lowner_test,5762,4435,37954,89.537380,86.819471,88.157482
1288,3,8000,15,train,0,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648
1291,3,8000,15,train,1,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648


In [5]:
from evaluations.utils import merge_mean_std, LATEX_METRIC_NAMES
size_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                 index=["timestep", "gazetteer_size"],
                                 aggfunc=["mean", "std"]).swaplevel(
                                     0, 1, axis=1).sort_index(axis=1)

metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_col in metric_names:
    column = (LATEX_METRIC_NAMES[metric_col],
                    "")
    new_columns.append(column)
    size_agg_df[column] = merge_mean_std(size_agg_df,
                                        (metric_col, "mean"),
                                        (metric_col, "std"))
size_agg_df["Dimension"] = "Gazetteer size"
size_agg_df.reset_index(inplace=True)
size_agg_df.set_index(["timestep", "Dimension", "gazetteer_size"], inplace=True)
size_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                            
timestep Dimension      gazetteer_size                                      
0        Gazetteer size 2000            85.05 (108)  87.60 (23)  82.68 (200)
                        4000            85.26 (155)  87.84 (18)  82.86 (279)
                        6000            85.64 (113)  87.98 (25)  83.43 (200)
                        8000             85.83 (89)  88.14 (21)  83.65 (162)
1        Gazetteer size 2000            85.07 (108)  87.61 (22)  82.69 (199)
                        4000            85.27 (153)  87.86 (19)  82.88 (277)
                        6000            85.66 (113)  88.00 (25)  83.46 (201)
                        8000             85.86 (88)  88.18 (21)  83.68 (160)
2        Gazetteer size 2000            86.77 (109)  88.90 (25)  84.76 (208)
                        4000            87.03 (154)  89.19 (18)  85.03 (282)
                        6000            87.46 (114)  89.35 (25)  85.67 (207)
                        8000             87.65 (91)  89.51 (23)  85.87 (164)

In [6]:
ratio_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                  index=["timestep", "error_ratio"],
                                  aggfunc=["mean", "std"]).swaplevel(
                                      0, 1, axis=1).sort_index(axis=1)
timesteps = [0, 1, 2]
metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_col in metric_names:
    column = (LATEX_METRIC_NAMES[metric_col], "")
    new_columns.append(column)
    ratio_agg_df[column] = merge_mean_std(ratio_agg_df,
                                            (metric_col, "mean"),
                                            (metric_col, "std"))
ratio_agg_df["Dimension"] = "Error ratio"
ratio_agg_df.reset_index(inplace=True)
ratio_agg_df.set_index(["timestep", "Dimension", "error_ratio"], inplace=True)
ratio_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                      
timestep Dimension   error_ratio                                      
0        Error ratio 0             86.25 (27)  87.94 (33)   84.62 (31)
                     5             85.69 (84)  87.91 (26)  83.60 (148)
                     10            85.38 (67)  87.90 (29)  83.01 (117)
                     15           84.45 (174)  87.82 (29)  81.39 (313)
1        Error ratio 0             86.25 (27)  87.94 (33)   84.62 (31)
                     5             85.70 (84)  87.92 (27)  83.61 (147)
                     10            85.41 (68)  87.92 (29)  83.04 (118)
                     15           84.50 (175)  87.87 (31)  81.44 (313)
2        Error ratio 0             88.05 (32)  89.29 (34)   86.85 (39)
                     5             87.48 (84)  89.24 (31)  85.80 (149)
                     10            87.16 (68)  89.27 (32)  85.16 (120)
                     15           86.22 (176)  89.16 (31)  83.52 (320)

In [7]:
part_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                 index=["timestep", "error_part"],
                                 aggfunc=["mean", "std"]).swaplevel(
                                     0, 1, axis=1).sort_index(axis=1)
metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_col in metric_names:
    column = (LATEX_METRIC_NAMES[metric_col], "")
    new_columns.append(column)
    part_agg_df[column] = merge_mean_std(part_agg_df,
                                            (metric_col, "mean"),
                                            (metric_col, "std"))
part_agg_df["Dimension"] = "Error part"
part_agg_df.reset_index(inplace=True)
part_agg_df.set_index(["timestep", "Dimension", "error_part"], inplace=True)
part_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                    
timestep Dimension  error_part                                      
0        Error part both        85.08 (126)  87.86 (29)  82.50 (224)
                    gazetteer    86.20 (30)  87.93 (30)   84.53 (38)
                    train       85.05 (139)  87.88 (29)  82.43 (248)
1        Error part both        85.12 (124)  87.89 (30)  82.54 (221)
                    gazetteer    86.23 (31)  87.97 (31)   84.56 (39)
                    train       85.05 (139)  87.88 (29)  82.43 (248)
2        Error part both        86.86 (127)  89.20 (33)  84.67 (228)
                    gazetteer    87.98 (33)  89.29 (33)   86.72 (42)
                    train       86.84 (142)  89.23 (31)  84.62 (258)

In [8]:
import pandas as pd

perf_df = pd.concat([
    size_agg_df[new_columns], ratio_agg_df[new_columns],
    part_agg_df[new_columns]
])
perf_df.sort_index(inplace=True)


In [9]:
perf_df.index.rename(["Timestep", "Dimension", "Value"], inplace=True)
perf_df.columns.rename(["Metric", None], inplace=True)
print(perf_df.style.to_latex(
    siunitx=True,
    hrules=True,
    multirow_align="t",
    column_format=
    "@{}lll*3{S[table-format = 2.2 (2)]}@{} "
))


\begin{tabular}{@{}lll*3{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {} & {Metric} & {Micro-F1} & {Precision} & {Recall} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Dimension} & {Value} & {} & {} & {} \\
\midrule
\multirow[t]{11}{*}{0} & \multirow[t]{3}{*}{Error part} & both & 85.08 (126) & 87.86 (29) & 82.50 (224) \\
 &  & gazetteer & 86.20 (30) & 87.93 (30) & 84.53 (38) \\
 &  & train & 85.05 (139) & 87.88 (29) & 82.43 (248) \\
 & \multirow[t]{4}{*}{Error ratio} & 0 & 86.25 (27) & 87.94 (33) & 84.62 (31) \\
 &  & 5 & 85.69 (84) & 87.91 (26) & 83.60 (148) \\
 &  & 10 & 85.38 (67) & 87.90 (29) & 83.01 (117) \\
 &  & 15 & 84.45 (174) & 87.82 (29) & 81.39 (313) \\
 & \multirow[t]{4}{*}{Gazetteer size} & 2000 & 85.05 (108) & 87.60 (23) & 82.68 (200) \\
 &  & 4000 & 85.26 (155) & 87.84 (18) & 82.86 (279) \\
 &  & 6000 & 85.64 (113) & 87.98 (25) & 83.43 (200) \\
 &  & 8000 & 85.83 (89) & 88.14 (21) & 83.65 (162) \\
\multirow[t]{11}{*}{1} & \multirow[t]{3}{*}{Error part} & both & 85.12 

## Second Performance table
1. (Data part = Both)
2. Timestep = 3
3. index = Data part, Error rate
4. columns = size
values = f1

In [10]:
from evaluations.content_02 import aggregate_performance_metrics

agg_df = aggregate_performance_metrics(metrics_df).reset_index()
agg_df = agg_df[agg_df["dataset"] == "lowner_test"]
agg_df

,seed,gazetteer_size,error_ratio,error_part,timestep,dataset,fn,fp,tp,precision,recall,f1
1,1,2000,0,both,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
4,1,2000,0,both,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
7,1,2000,0,both,2,lowner_test,5826,4898,37890,88.552865,86.673072,87.602885
10,1,2000,0,gazetteer,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
13,1,2000,0,gazetteer,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,3,8000,15,gazetteer,1,lowner_test,6626,4930,37090,88.267492,84.843078,86.521415
1285,3,8000,15,gazetteer,2,lowner_test,5762,4435,37954,89.537380,86.819471,88.157482
1288,3,8000,15,train,0,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648
1291,3,8000,15,train,1,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648


In [11]:
from evaluations.utils import merge_mean_std

perf_df = agg_df.pivot_table(
    values="f1",
    index=["timestep", "gazetteer_size", "error_part"],
    columns="error_ratio",
    aggfunc=["mean",
             "std"]).swaplevel(0, 1,
                               axis=1).sort_index(axis=1).sort_index(axis=0)

ratios = [0, 5, 10, 15]
new_columns = []
for ratio in ratios:
    column = (ratio, "")
    new_columns.append(column)
    perf_df[column] = merge_mean_std(perf_df, (ratio, "mean"), (ratio, "std"))

perf_df = perf_df[new_columns]
perf_df.index.rename(["Timestep", "Gazetteer size", "Error part"], inplace=True)
perf_df.columns.rename(["Error ratio", None], inplace=True)
perf_df.sort_index(inplace=True)
print(
    perf_df.style.to_latex(
        siunitx=True,
        hrules=True,
        multirow_align="t",
        column_format="@{}ll*4{S[table-format = 2.2 (2)]}@{} "))


\begin{tabular}{@{}ll*4{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {} & {Error ratio} & {0} & {5} & {10} & {15} \\
{} & {} & {} & {} & {} & {} & {} \\
{Timestep} & {Gazetteer size} & {Error part} & {} & {} & {} & {} \\
\midrule
\multirow[t]{12}{*}{0} & \multirow[t]{3}{*}{2000} & both & 85.86 (5) & 83.98 (232) & 84.59 (13) & 83.93 (28) \\
 &  & gazetteer & 85.86 (5) & 85.77 (21) & 85.92 (8) & 85.76 (23) \\
 &  & train & 85.86 (5) & 85.29 (19) & 84.46 (12) & 83.35 (55) \\
 & \multirow[t]{3}{*}{4000} & both & 86.24 (10) & 85.54 (31) & 85.02 (15) & 82.66 (228) \\
 &  & gazetteer & 86.24 (10) & 86.11 (11) & 85.91 (19) & 86.24 (3) \\
 &  & train & 86.24 (10) & 85.69 (2) & 84.70 (25) & 82.47 (245) \\
 & \multirow[t]{3}{*}{6000} & both & 86.40 (9) & 85.69 (30) & 85.08 (9) & 84.40 (21) \\
 &  & gazetteer & 86.40 (9) & 86.22 (9) & 86.42 (12) & 86.30 (11) \\
 &  & train & 86.40 (9) & 85.83 (12) & 85.22 (13) & 83.26 (239) \\
 & \multirow[t]{3}{*}{8000} & both & 86.49 (26) & 85.86 (40) & 85.37 (5

# Collect data metrics

## Collect gazetteer data

In [1]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.content_02 import get_search_results_entity_coverage_per_sample

eecr_search_results = get_search_results_entity_coverage_per_sample()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
eecr_search_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21794400 entries, 0 to 21794399
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   seed            int64  
 1   gazetteer_size  int64  
 2   error_ratio     int64  
 3   error_part      object 
 4   timestep        int64  
 5   dataset         object 
 6   doc_id          object 
 7   targets         int64  
 8   eecr            float64
 9   ρ=1             int64  
 10  ρ ∈ (0.5,1)     int64  
 11  ρ ∈ (0,0.5]     int64  
 12  ρ=0∧C≠0         int64  
 13  ρ=0∧C=0         int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 2.3+ GB


In [4]:
from evaluations.content_02 import get_search_results_data_ccr_metrics

ccr_search_results = get_search_results_data_ccr_metrics()

In [5]:
ccr_search_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21794400 entries, 0 to 21794399
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   seed            int64  
 1   gazetteer_size  int64  
 2   error_ratio     int64  
 3   error_part      object 
 4   timestep        int64  
 5   dataset         object 
 6   doc_id          object 
 7   total entities  int64  
 8   search score    float64
 9   max             float64
 10  φ ∈ (0.5,1]     int64  
 11  φ ∈ (0,0.5]     int64  
 12  φ ∈ (-0.5,0]    int64  
 13  φ ∈ [-1,-0.5]   int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 2.3+ GB


## EECR table

In [6]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)

In [7]:
from evaluations.content_02 import get_labeled_data_entity_coverage, get_labeled_data_entity_coverage_per_sample, get_search_results_entity_coverage_per_sample

eecr_labeled_data_df = get_labeled_data_entity_coverage()
eecr_labeled_data_per_sample_df = get_labeled_data_entity_coverage_per_sample()
eecr_search_results_per_sample_df = get_search_results_entity_coverage_per_sample()

In [8]:
eecr_labeled_data_df = eecr_labeled_data_df[eecr_labeled_data_df["dataset"] ==
                                            "lowner_test"]
eecr_labeled_data_per_sample_df = eecr_labeled_data_per_sample_df[
    eecr_labeled_data_per_sample_df["dataset"] == "lowner_test"]
eecr_search_results_per_sample_df = eecr_search_results_per_sample_df[
    eecr_search_results_per_sample_df["dataset"] == "lowner_test"]


In [9]:
from evaluations.utils import LATEX_METRIC_NAMES, merge_mean_std
import pandas as pd
from typing import List
dimensions = ["gazetteer_size", "error_ratio", "error_part"]
dimension_name = {
    "gazetteer_size": "Gazetteer size",
    "error_ratio": "Error ratio", "error_part": "Error part"
}
metric_columns = [
    "eecr_labeled_data", "eecr_labeled_data_per_sample", "eecr_search_results"
]
eecr_labeled_data_df.rename({"eecr": "eecr_labeled_data"},
                            axis=1,
                            inplace=True)
eecr_labeled_data_per_sample_df.rename({"eecr": "eecr_labeled_data_per_sample"},
                            axis=1,
                            inplace=True)
eecr_search_results_per_sample_df.rename({"eecr": "eecr_search_results"},
                            axis=1,
                            inplace=True)



dfs = [
    eecr_labeled_data_df, eecr_labeled_data_per_sample_df,
    eecr_search_results_per_sample_df
]

eecr_table_columns: List[pd.DataFrame] = []

for df in dfs:
    filtered_metric_columns = [met_col for met_col in metric_columns if met_col in df.columns.tolist()]
    dim_dfs = []
    for dimension in dimensions:
        dim_agg_df = df.pivot_table(values=filtered_metric_columns,
                                        index=["timestep", dimension],
                                        aggfunc=["mean", "std"]).swaplevel(
                                            0, 1, axis=1).sort_index(axis=1)

        new_columns = []
        for metric_col in filtered_metric_columns:
            column = (LATEX_METRIC_NAMES[metric_col], "")
            new_columns.append(column)
            dim_agg_df[column] = merge_mean_std(dim_agg_df, (metric_col, "mean"),
                                                (metric_col, "std"), 4)
        dim_agg_df["Dimension"] = dimension_name[dimension]
        dim_agg_df.reset_index(inplace=True)
        dim_agg_df.set_index(["timestep", "Dimension", dimension],
                            inplace=True)
        dim_dfs.append(dim_agg_df[new_columns])
    tmp_table = pd.concat(dim_dfs).sort_index()
    tmp_table.index.rename(["Timestep", "Dimension", "Value"], inplace=True)
    tmp_table.columns.rename(["Metric", None], inplace=True)
    eecr_table_columns.append(tmp_table)


print(eecr_table_columns[0].join(eecr_table_columns[1:]).style.to_latex(
    siunitx=True,
    hrules=True,
    multirow_align="t",
    column_format="@{}lll*3{S[table-format = 2.2 (2)]}@{} "))


\begin{tabular}{@{}lll*3{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {} & {Metric} & {EECR labeled data} & {EECR labeled data per sample} & {EECR search results} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Dimension} & {Value} & {} & {} & {} \\
\midrule
\multirow[t]{11}{*}{0} & \multirow[t]{3}{*}{Error part} & both & 0.2780 (536) & 0.2248 (3694) & 0.1670 (3338) \\
 &  & gazetteer & 0.2780 (536) & 0.2248 (3694) & 0.1670 (3338) \\
 &  & train & 0.2821 (541) & 0.2282 (3720) & 0.1715 (3376) \\
 & \multirow[t]{4}{*}{Error ratio} & 0 & 0.2815 (551) & 0.2278 (3719) & 0.1715 (3375) \\
 &  & 5 & 0.2807 (540) & 0.2272 (3712) & 0.1696 (3362) \\
 &  & 10 & 0.2799 (529) & 0.2260 (3702) & 0.1677 (3343) \\
 &  & 15 & 0.2754 (538) & 0.2228 (3678) & 0.1653 (3321) \\
 & \multirow[t]{4}{*}{Gazetteer size} & 2000 & 0.1998 (47) & 0.1610 (3219) & 0.1219 (2911) \\
 &  & 4000 & 0.2670 (31) & 0.2156 (3630) & 0.1607 (3280) \\
 &  & 6000 & 0.3086 (48) & 0.2498 (3836) & 0.1859 (3483) \\
 &  & 8000 & 0.3420 (

## CCR table

In [10]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)

In [11]:
from evaluations.content_02 import get_search_results_data_ccr_metrics

search_results_ccr = get_search_results_data_ccr_metrics()
search_results_ccr.columns

Index(['seed', 'gazetteer_size', 'error_ratio', 'error_part', 'timestep',
       'dataset', 'doc_id', 'total entities', 'search score', 'max',
       'φ ∈ (0.5,1]', 'φ ∈ (0,0.5]', 'φ ∈ (-0.5,0]', 'φ ∈ [-1,-0.5]'],
      dtype='object')

In [12]:
from evaluations.utils import LATEX_METRIC_NAMES, merge_mean_std
import pandas as pd
from typing import List

dimensions = ["gazetteer_size", "error_ratio", "error_part"]
dimension_name = {
    "gazetteer_size": "Gazetteer size",
    "error_ratio": "Error ratio",
    "error_part": "Error part"
}
metric_columns = [
    "max", "search score", "total entities"
]

df = search_results_ccr[search_results_ccr["dataset"]=="lowner_test"]

filtered_metric_columns = [
    met_col for met_col in metric_columns
    if met_col in df.columns.tolist()
]
dim_dfs = []
for dimension in dimensions:
    dim_agg_df = df.pivot_table(values=filtered_metric_columns,
                                index=["timestep", dimension],
                                aggfunc=["mean", "std"]).swaplevel(
                                    0, 1, axis=1).sort_index(axis=1)

    new_columns = []
    for metric_col in filtered_metric_columns:
        column = (LATEX_METRIC_NAMES[metric_col], "")
        new_columns.append(column)
        dim_agg_df[column] = merge_mean_std(dim_agg_df,
                                            (metric_col, "mean"),
                                            (metric_col, "std"), 4)
    dim_agg_df["Dimension"] = dimension_name[dimension]
    dim_agg_df.reset_index(inplace=True)
    dim_agg_df.set_index(["timestep", "Dimension", dimension],
                            inplace=True)
    dim_dfs.append(dim_agg_df[new_columns])
tmp_table = pd.concat(dim_dfs).sort_index()
tmp_table.index.rename(["Timestep", "Dimension", "Value"], inplace=True)
tmp_table.columns.rename(["Metric", None], inplace=True)
tmp_table.sort_index(inplace=True)

print(
    tmp_table.style.to_latex(
        siunitx=True,
        hrules=True,
        multirow_align="t",
        column_format="@{}lll*3{S[table-format = 2.2 (2)]}@{} "))


\begin{tabular}{@{}lll*3{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {} & {Metric} & {Top-1 Cosine Similarity} & {Search score} & {Retrieved entities} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Dimension} & {Value} & {} & {} & {} \\
\midrule
\multirow[t]{11}{*}{0} & \multirow[t]{3}{*}{Error part} & both & 0.5498 (1139) & 4.0953 (18054) & 10.2549 (24091) \\
 &  & gazetteer & 0.5498 (1139) & 4.0953 (18054) & 10.2549 (24091) \\
 &  & train & 0.5508 (1140) & 4.0599 (18371) & 10.1525 (24511) \\
 & \multirow[t]{4}{*}{Error ratio} & 0 & 0.5508 (1140) & 4.0720 (18597) & 10.1661 (24769) \\
 &  & 5 & 0.5505 (1143) & 4.0681 (18071) & 10.1810 (24073) \\
 &  & 10 & 0.5498 (1139) & 4.0750 (17997) & 10.2241 (23967) \\
 &  & 15 & 0.5492 (1137) & 4.1191 (17967) & 10.3118 (24104) \\
 & \multirow[t]{4}{*}{Gazetteer size} & 2000 & 0.5214 (1096) & 3.8237 (16646) & 10.2331 (23280) \\
 &  & 4000 & 0.5466 (1123) & 4.0566 (17919) & 10.2263 (24041) \\
 &  & 6000 & 0.5612 (1135) & 4.1828 (18410) & 10.2106 

## Correlation table

Perf metrics: Precision, Recall, F1 + Input Entity count

In [1]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)

In [2]:
from evaluations.content_02 import get_per_sample_metrics, get_search_results_data_ccr_metrics, get_labeled_data_entity_coverage_per_sample, get_search_results_entity_coverage_per_sample

performance_metrics = get_per_sample_metrics()



/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
performance_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130766400 entries, 0 to 130766399
Data columns (total 18 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   seed            int64 
 1   checkpoint      object
 2   gazetteer_size  int64 
 3   error_ratio     int64 
 4   error_part      object
 5   timestep        int64 
 6   dataset         object
 7   doc_id          object
 8   targets         int64 
 9   entity_type     object
 10  tp              int64 
 11  fn              int64 
 12  fp              int64 
 13  error_type1     int64 
 14  error_type2     int64 
 15  error_type3     int64 
 16  error_type4     int64 
 17  error_type5     int64 
dtypes: int64(13), object(5)
memory usage: 17.5+ GB


In [4]:
error_rate = performance_metrics[
    (performance_metrics["checkpoint"] == "last")].pivot_table(
        index=[
            "seed", "gazetteer_size", "error_ratio", "error_part", "timestep", "dataset", "doc_id"
        ],
        values=[
            "tp", "fp", "fn", "error_type1", "error_type2", "error_type3",
            "error_type4", "error_type5", "targets"
        ],
        aggfunc="sum")


In [5]:
del performance_metrics

In [6]:

error_rate["error rate"] = error_rate["fp"] + error_rate["fn"]
error_rate["precision"] = (error_rate["tp"]) / (error_rate["tp"] +
                                                error_rate["fp"] + 1e-8)
error_rate["recall"] = (error_rate["tp"]) / (error_rate["tp"] +
                                             error_rate["fn"] + 1e-8)
error_rate["f1"] = 2 * error_rate["precision"] * error_rate["recall"] / (
    error_rate["precision"] + error_rate["recall"] + 1e-8)
error_rate.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21794400 entries, (1, 2000, 0, 'both', 0, 'lowner_dev', 'lowner_dev_0') to (3, 8000, 15, 'train', 2, 'lowner_train', 'lowner_train_9999')
Data columns (total 13 columns):
 #   Column       Dtype  
---  ------       -----  
 0   error_type1  int64  
 1   error_type2  int64  
 2   error_type3  int64  
 3   error_type4  int64  
 4   error_type5  int64  
 5   fn           int64  
 6   fp           int64  
 7   targets      int64  
 8   tp           int64  
 9   error rate   int64  
 10  precision    float64
 11  recall       float64
 12  f1           float64
dtypes: float64(3), int64(10)
memory usage: 2.3+ GB


In [7]:
search_results_ccr = get_search_results_data_ccr_metrics()
eecr_labeled_data_per_sample_df = get_labeled_data_entity_coverage_per_sample()
eecr_search_results_per_sample_df = get_search_results_entity_coverage_per_sample(
)

In [8]:
eecr_labeled_data_per_sample_df.rename(
    {"eecr": "eecr_labeled_data_per_sample"}, axis=1, inplace=True)
eecr_search_results_per_sample_df.rename({"eecr": "eecr_search_results"},
                                         axis=1,
                                         inplace=True)


In [9]:
search_results_ccr.set_index([
    "seed", "gazetteer_size", "error_ratio", "error_part", "timestep",
    "dataset", "doc_id"
],
                             inplace=True)
search_results_ccr.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21794400 entries, (1, 2000, 0, 'train', 0, 'lowner_train', 'lowner_train_0') to (3, 8000, 15, 'both', 0, 'lowner_dev', 'lowner_dev_3365')
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   total entities  int64  
 1   search score    float64
 2   max             float64
 3   φ ∈ (0.5,1]     int64  
 4   φ ∈ (0,0.5]     int64  
 5   φ ∈ (-0.5,0]    int64  
 6   φ ∈ [-1,-0.5]   int64  
dtypes: float64(2), int64(5)
memory usage: 1.3+ GB


In [10]:
eecr_labeled_data_per_sample_df.set_index([
    "seed", "gazetteer_size", "error_ratio", "error_part", "timestep",
    "dataset", "doc_id"
],
                                          inplace=True)
eecr_labeled_data_per_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21794400 entries, (1, 2000, 0, 'train', 0, 'lowner_train', 'lowner_train_0') to (3, 8000, 15, 'both', 2, 'lowner_test', 'lowner_test_33660')
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   targets                       int64  
 1   eecr_labeled_data_per_sample  float64
 2   ρ=1                           int64  
 3   ρ ∈ (0.5,1)                   int64  
 4   ρ ∈ (0,0.5]                   int64  
 5   ρ=0∧C≠0                       int64  
 6   ρ=0∧C=0                       int64  
dtypes: float64(1), int64(6)
memory usage: 1.3+ GB


In [11]:
eecr_search_results_per_sample_df.set_index([
    "seed", "gazetteer_size", "error_ratio", "error_part", "timestep",
    "dataset", "doc_id"
],
                                            inplace=True)
eecr_search_results_per_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21794400 entries, (1, 2000, 0, 'train', 0, 'lowner_train', 'lowner_train_0') to (3, 8000, 15, 'both', 2, 'lowner_test', 'lowner_test_33660')
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   targets              int64  
 1   eecr_search_results  float64
 2   ρ=1                  int64  
 3   ρ ∈ (0.5,1)          int64  
 4   ρ ∈ (0,0.5]          int64  
 5   ρ=0∧C≠0              int64  
 6   ρ=0∧C=0              int64  
dtypes: float64(1), int64(6)
memory usage: 1.3+ GB


In [12]:
combined_df = error_rate.join([
    eecr_labeled_data_per_sample_df["eecr_labeled_data_per_sample"],
    eecr_search_results_per_sample_df["eecr_search_results"],
    search_results_ccr[["max", "search score"]]
])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21794400 entries, (1, 2000, 0, 'both', 0, 'lowner_dev', 'lowner_dev_0') to (3, 8000, 15, 'train', 2, 'lowner_train', 'lowner_train_9999')
Data columns (total 17 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   error_type1                   int64  
 1   error_type2                   int64  
 2   error_type3                   int64  
 3   error_type4                   int64  
 4   error_type5                   int64  
 5   fn                            int64  
 6   fp                            int64  
 7   targets                       int64  
 8   tp                            int64  
 9   error rate                    int64  
 10  precision                     float64
 11  recall                        float64
 12  f1                            float64
 13  eecr_labeled_data_per_sample  float64
 14  eecr_search_results           float64
 15  max                           float64
 16  

In [13]:
combined_df.head()

error_type1  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                         
1    2000           0           both       0        lowner_dev lowner_dev_0               0   
                                                               lowner_dev_1               0   
                                                               lowner_dev_10              0   
                                                               lowner_dev_100             1   
                                                               lowner_dev_1000            0   

                                                                                error_type2  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                         
1    2000           0           both       0        lowner_dev lowner_dev_0               0   
                                                               lowner_dev_1               0   
                                                               lowner_dev_10              0   
                                                               lowner_dev_100             0   
                                                               lowner_dev_1000            0   

                                                                                error_type3  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                         
1    2000           0           both       0        lowner_dev lowner_dev_0               0   
                                                               lowner_dev_1               0   
                                                               lowner_dev_10              0   
                                                               lowner_dev_100             0   
                                                               lowner_dev_1000            0   

                                                                                error_type4  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                         
1    2000           0           both       0        lowner_dev lowner_dev_0               0   
                                                               lowner_dev_1               0   
                                                               lowner_dev_10              0   
                                                               lowner_dev_100             0   
                                                               lowner_dev_1000            0   

                                                                                error_type5  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                         
1    2000           0           both       0        lowner_dev lowner_dev_0               0   
                                                               lowner_dev_1               0   
                                                               lowner_dev_10              0   
                                                               lowner_dev_100             0   
                                                               lowner_dev_1000            0   

                                                                                fn  \
seed gazetteer_size error_ratio error_part timestep dataset    doc_id                
1    2000           0           both       0        lowner_dev lowner_dev_0      0   
                                                               lowner_dev_1      0   
                                                               lowner_dev_10     0   
                                                               lowner_dev_100    0   
                                                               lowner_dev_1000   0   

                                                                                fp  \
seed gazetteer_size error_ratio error_part timestep dataset  

In [14]:
combined_df.to_parquet(
    os.path.join(thesis_path, "evaluations", "metrics",
                 "02_content_combined_df.parquet"))


In [15]:
from scipy.stats import pearsonr
from typing import List
import pandas as pd


def per_sample_pearsonr(df: pd.DataFrame,
                        data_metrics: List[str],
                        performance_metrics: List[str],
                        alternative: str = "two-sided",
                        confidence_level: float = 0.95):
    no_index_df = df.reset_index()
    timesteps = [0, 1, 2]
    dimensions = ["gazetteer_size", "error_ratio", "error_part"]
    datasets = no_index_df["dataset"].unique()

    pearson_corrs = []
    for timestep in timesteps:
        timestep_df = no_index_df[no_index_df["timestep"]==timestep]
        for dataset in datasets:
            dataset_df = timestep_df[timestep_df["dataset"] == dataset]
            for dim in dimensions:
                values = dataset_df[dim].unique().tolist()
                for value in values:
                    model_df = dataset_df[dataset_df[dim] == value]
                    for data_metric in data_metrics:
                        metric_df = model_df.fillna(value=0.0)
                        for perf_metric in performance_metrics:
                            corr = pearsonr(metric_df[data_metric].tolist(),
                                            metric_df[perf_metric].tolist(),
                                            alternative=alternative)
                            conf = corr.confidence_interval(
                                confidence_level=confidence_level)
                            pearson_corrs.append({
                                "timestep": timestep,
                                "dimension": dim,
                                "value": value,
                                "dataset": dataset,
                                "perf_metric": perf_metric,
                                "data_metric": data_metric,
                                "coefficient": corr.statistic,
                                "pvalue": corr.pvalue,
                                "conf_low": conf.low,
                                "conf_high": conf.high
                            })

    return pd.DataFrame.from_records(pearson_corrs)


pearson_corr_df = per_sample_pearsonr(combined_df, [
    "eecr_labeled_data_per_sample", "eecr_search_results", "max",
    "search score"
], [
    "error_type1", "error_type2", "error_type3", "error_type4", "error_type5",
    "fn", "fp", "error rate", "tp", "precision", "recall", "f1", "targets"
])
pearson_corr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestep     5148 non-null   int64  
 1   dimension    5148 non-null   object 
 2   value        5148 non-null   object 
 3   dataset      5148 non-null   object 
 4   perf_metric  5148 non-null   object 
 5   data_metric  5148 non-null   object 
 6   coefficient  5148 non-null   float64
 7   pvalue       5148 non-null   float64
 8   conf_low     5148 non-null   float64
 9   conf_high    5148 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 402.3+ KB


In [16]:
pearson_corr_df.to_pickle(
    os.path.join(thesis_path, "evaluations", "metrics",
                 "02_content_correlations.pkl.tar.gz"))


In [17]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)

In [18]:
import pandas as pd

pearson_corr_df = pd.read_pickle(
    os.path.join(thesis_path, "evaluations", "metrics",
                 "02_content_correlations.pkl.tar.gz"))

In [19]:
df = pearson_corr_df[(pearson_corr_df["dataset"] == "lowner_test")
                & (pearson_corr_df["timestep"] == 2)
                & (pearson_corr_df["perf_metric"].isin(
                    ["f1", "recall", "targets"]))].pivot_table(
                        values="coefficient",
                        columns="data_metric",
                        index=["perf_metric", "dimension", "value"])


In [20]:
from evaluations.utils import METRIC_ORDER, MODEL_ORDER, LATEX_METRIC_NAMES, highlight_correlations

dimension_names = {
    "error_part": "Error part",
    "error_ratio": "Error ratio",
    "gazetteer_size": "Gazetteer size"
}

def index_order(x: pd.Index):
    if x.name == "perf_metric":
        return x.map(METRIC_ORDER)
    return x

def column_order(x: pd.Index):
    if x.name == "data_metric":
        return x.map(METRIC_ORDER)
    return x


df.sort_index(axis=0, inplace=True, key=index_order)
df.sort_index(axis=1, inplace=True, key=column_order)
df.index = df.index.map(lambda x:
                        (LATEX_METRIC_NAMES[x[0]], dimension_names[x[1]], x[2]))
df.columns = df.columns.map(LATEX_METRIC_NAMES)
df.index.rename(["Performance Metric", "Dimension", "Value"], inplace=True)
df.columns.rename("Input Data Metric", inplace=True)

print(df.style.pipe(highlight_correlations).format('{:.4f}').to_latex(
    siunitx=True,
    hrules=True,
    multirow_align="t",
    column_format=
    "@{}lll*4{S[table-format = 1.4, group-minimum-digits=5]}@{}"))

\begin{tabular}{@{}lll*4{S[table-format = 1.4, group-minimum-digits=5]}@{}}
\toprule
{} & {} & {Input Data Metric} & {EECR labeled data per sample} & {EECR search results} & {Top-1 Cosine Similarity} & {Search score} \\
{Performance Metric} & {Dimension} & {Value} & {} & {} & {} & {} \\
\midrule
\multirow[t]{11}{*}{Recall} & \multirow[t]{3}{*}{Error part} & both & \cellcolor[HTML]{6BAED6} 0.6456 & \cellcolor[HTML]{C6DBEF} 0.4745 & 0.2579 & 0.2638 \\
 &  & gazetteer & \cellcolor[HTML]{6BAED6} 0.6668 & \cellcolor[HTML]{C6DBEF} 0.4920 & 0.2695 & 0.2717 \\
 &  & train & \cellcolor[HTML]{6BAED6} 0.6445 & \cellcolor[HTML]{C6DBEF} 0.4741 & 0.2578 & 0.2637 \\
 & \multirow[t]{4}{*}{Error ratio} & 0 & \cellcolor[HTML]{6BAED6} 0.6694 & \cellcolor[HTML]{C6DBEF} 0.4948 & 0.2697 & 0.2720 \\
 &  & 5 & \cellcolor[HTML]{6BAED6} 0.6574 & \cellcolor[HTML]{C6DBEF} 0.4844 & 0.2632 & 0.2679 \\
 &  & 10 & \cellcolor[HTML]{6BAED6} 0.6526 & \cellcolor[HTML]{C6DBEF} 0.4815 & 0.2629 & 0.2677 \\
 &  & 15 & \cellc

# Dataset statistics
1. Statistics like in Dataset part for each Error rate on train + dev
2. Error types distribution for train + dev

# Gazetteer statistics
1. Statistics like in experiment 01 for each size + error rate
2. Error types distribution for each size + error rate